In [1]:
import torch.nn as nn
import timm
from PIL import Image
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import utils
import cv2
from torchvision import datasets, models

In [2]:
import albumentations as A

In [3]:
import numpy as np
import pandas as pd
import torch
import os
import matplotlib.pyplot as plt
import torch.nn.functional as F 
from torch import nn,optim
from torchvision import transforms as T,datasets,models
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from collections import OrderedDict
import tqdm
from torch import nn, optim
from torch.autograd import Variable

In [4]:
data_dir = os.listdir("Y:\Python\Datasets\chest_xray")
path = "Y:\Python\Datasets\chest_xray"

In [5]:
train_n = os.path.join(path, "train", "NORMAL")
train_p = os.path.join(path, "train", "PNEUMONIA")

val_n = os.path.join(path, "val", "NORMAL")
val_p = os.path.join(path, "val", "PNEUMONIA")

test_n = os.path.join(path, "test", "NORMAL")
test_p = os.path.join(path, "test", "PNEUMONIA")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 46-47: malformed \N character escape (<ipython-input-6-c8391733f18d>, line 1)

In [7]:
def create_X_y(data_n, data_p):
    norm_img_list = []
    for filename in os.listdir(data_n):
        norm_img_list.append(os.path.join(data_n, filename))
        
    pneu_img_list = []
    for filename in os.listdir(data_p):
        pneu_img_list.append(os.path.join(data_p, filename))
    
    data = np.concatenate([norm_img_list, pneu_img_list])
    target = np.zeros(len(data))
    target[:len(norm_img_list)] = 1
    
    return data, target

In [8]:
train_X, train_y = create_X_y(train_n, train_p)
val_X, val_y = create_X_y(val_n, val_p)
test_X, test_y = create_X_y(test_n, test_p)

In [10]:
train_X

array(['Y:\\Python\\Datasets\\chest_xray\\train\\NORMAL\\IM-0115-0001.jpeg',
       'Y:\\Python\\Datasets\\chest_xray\\train\\NORMAL\\IM-0117-0001.jpeg',
       'Y:\\Python\\Datasets\\chest_xray\\train\\NORMAL\\IM-0119-0001.jpeg',
       ...,
       'Y:\\Python\\Datasets\\chest_xray\\train\\PNEUMONIA\\person9_bacteria_39.jpeg',
       'Y:\\Python\\Datasets\\chest_xray\\train\\PNEUMONIA\\person9_bacteria_40.jpeg',
       'Y:\\Python\\Datasets\\chest_xray\\train\\PNEUMONIA\\person9_bacteria_41.jpeg'],
      dtype='<U75')

In [ ]:
X

In [10]:
def data_transforms():
    transformed_data = T.Compose([
                            
                            T.RandomRotation(22),
                            T.Grayscale(),
                            T.Resize(size = (256,256)),
                            T.ToTensor()
                            
                            
])
    return transformed_data

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
TEST = "test"
TRAIN = "train"
VAL = "val"

In [ ]:
train_norm_data = os.listdir(train_n)
train_p_data = os.listdir(train_p)

In [ ]:
        np.array(Image.open(train_X[21])).shape

In [13]:
class LungsDataset(Dataset):
    def __init__(self, data, target, transform = None):
        super().__init__()
        self.data = data
        self.target = target
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        y = self.target[index]
        img = Image.open(self.data[index])
        
        if self.transform is not None:
            img = self.transform(img)
        img = img/255
        return img, y

In [16]:
for X, y in train_dataloader:
    break

In [15]:
train_dataset = LungsDataset(train_X, train_y, data_transforms())
train_dataloader = DataLoader(train_dataset, batch_size = 8, shuffle = True)

val_dataset = LungsDataset(val_X, val_y, data_transforms())
val_dataloader = DataLoader(val_dataset, batch_size = 8, shuffle = True)

test_dataset = LungsDataset(test_X, test_y, data_transforms())
test_dataloader = DataLoader(test_dataset, batch_size = 2, shuffle = True)

In [ ]:
X[1]

In [ ]:
model = timm.create_model("res2next50", num_classes = 2, in_chans = 1).to(device)

In [ ]:
class PneumoniaModel(nn.Module):
    def __init__(self):
        super(PneumoniaModel, self).__init__()
        self.backbone = timm.create_model("tf_efficientnet_b0", in_chans = 1)
        self.backbone.classifier = nn.Linear(self.backbone.classifier.in_features, 2)
                                        
    def forward(self, X):
        X = self.backbone(X)
        return X

In [ ]:

model

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
num_epochs = 5


In [ ]:
from tqdm.notebook import tqdm

In [ ]:
torch.cuda.is_available()

In [ ]:
train_losses = []
val_losses = []


if torch.cuda.is_available():
    model.cuda()
    model.train()
    for e in range(1):
        i = 0
        for X, y in tqdm(train_dataloader):
            optimizer.zero_grad()
            pred = model(X.to(device))
            loss = loss_fn(pred, y.to(device).long())
            loss.backward()
            optimizer.step()
            
            train_losses.append(loss.item())
        
        for X, y in tqdm(val_dataloader):
            pred = model(X.to(device))
            loss = loss_fn(pred, y.to(device).long())
        
            val_losses.append(loss.item())
        
        i += 1   
        
        print("Epoch: ", i)
        print("train loss: ", np.mean(train_losses))
        print("val loss: ", np.mean(val_losses))

In [ ]:

all_pred = []
all_target = []
for X, y in tqdm(test_dataloader):
    pred = model(X.to(device))
    all_pred.append(torch.argmax(pred, dim = 1).detach().cpu().numpy())
    all_target.append(y.detach().cpu().numpy())
    


In [ ]:
all_pred = np.concatenate(all_pred)
all_target = np.concatenate(all_target)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
report = classification_report(all_target, all_pred)

In [ ]:
print(report)

In [ ]:
torch.argmax(pred, dim = 1) 

In [ ]:
y

In [ ]:
X.shape

In [ ]:
for X, y in train_dataloader:
    break